In [ ]:
import cv2
import numpy as np
from sklearn.cluster import KMeans

def track_volleyball(video_path):
    cap = cv2.VideoCapture(video_path)
    
    # Initialize background subtractor
    backSub = cv2.createBackgroundSubtractorMOG2()
    
    while(cap.isOpened()):
        ret, frame = cap.read()
        if not ret:
            break
        
        # Apply background subtraction
        fgMask = backSub.apply(frame)
        
        # Apply clustering to separate players from the background
        player_mask = cluster_players(frame)
        
        # Combine foreground mask with player mask
        fgMask = cv2.bitwise_and(fgMask, player_mask)
        
        # Apply HSV color space analysis for volleyball detection
        hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        ball_mask = detect_volleyball_hsv(hsv_frame)
        
        # Apply contour detection to find the volleyball
        ball_center = detect_volleyball(frame, fgMask, ball_mask)
        
        # Draw circle around the volleyball
        if ball_center is not None:
            cv2.circle(frame, ball_center, 10, (0, 255, 0), 2)
        
        # Display the frame
        cv2.imshow('Volleyball Tracking', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

def cluster_players(frame, num_clusters=2):
    # Convert frame to HSV color space
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # Extract saturation channel to enhance player colors
    saturation_channel = hsv_frame[:, :, 1]
    
    # Reshape saturation channel for clustering
    reshaped_saturation = saturation_channel.reshape((-1, 1))
    
    # Apply K-means clustering
    kmeans = KMeans(n_clusters=num_clusters, n_init=5)  # Adjust n_init value
    kmeans.fit(reshaped_saturation)
    
    # Extract player mask from clustering results
    player_mask = kmeans.labels_.reshape(saturation_channel.shape)
    
    # Convert player mask to binary mask
    player_mask = np.uint8(player_mask == 1) * 255
    
    return player_mask

def detect_volleyball_hsv(hsv_frame):
    # Define HSV range for volleyball color (adjust as needed)
    lower_bound = np.array([20, 100, 100])
    upper_bound = np.array([30, 255, 255])
    
    # Threshold HSV frame to detect volleyball
    ball_mask = cv2.inRange(hsv_frame, lower_bound, upper_bound)
    
    return ball_mask

def detect_volleyball(frame, fgMask, ball_mask):
    # Combine foreground mask with volleyball mask
    fgMask = cv2.bitwise_and(fgMask, ball_mask)
    
    # Find contours in the combined mask
    contours, _ = cv2.findContours(fgMask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Process contours to track the volleyball
    for contour in contours:
        # Filter contours based on area
        area = cv2.contourArea(contour)
        if 10< area < 100:  # Adjust area threshold as needed
            # Find the minimum enclosing circle for the contour
            (x, y), radius = cv2.minEnclosingCircle(contour)
            center = (int(x), int(y))
            
            # Return center of the volleyball
            return center
    
    return None

# Replace 'volleyball_match.mp4' with the path to your video file
track_volleyball("/home/gllaizz/Downloads/volleyball_match.mp4")

In [6]:
import cv2
import numpy as np
from sklearn.cluster import KMeans

def count_players(video_path, frame_skip=1):
    cap = cv2.VideoCapture(video_path)
    
    # Initialize background subtractor
    backSub = cv2.createBackgroundSubtractorMOG2()
    
    player_count = []
    frame_count = 0
    
    while(cap.isOpened()):
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_count += 1
        if frame_count % frame_skip != 0:
            continue  # Skip frames
        
        # Apply background subtraction
        fgMask = backSub.apply(frame)
        
        # Apply clustering to separate players from the background
        player_mask, num_players = cluster_players(frame)
        
        # Count the number of players detected in the frame
        player_count.append(num_players)
        
        # Display the frame
        cv2.imshow('Player Count', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

    # Print the total number of players detected throughout the video
    print("Total number of players detected:", sum(player_count))

def cluster_players(frame, num_clusters=2):
    # Convert frame to HSV color space
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # Extract saturation channel to enhance player colors
    saturation_channel = hsv_frame[:, :, 1]
    
    # Reshape saturation channel for clustering
    reshaped_saturation = saturation_channel.reshape((-1, 1))
    
    # Apply K-means clustering
    kmeans = KMeans(n_clusters=num_clusters, n_init=5)  # Adjust n_init value
    kmeans.fit(reshaped_saturation)
    
    # Extract player mask from clustering results
    player_mask = kmeans.labels_.reshape(saturation_channel.shape)
    
    # Convert player mask to binary mask
    player_mask = np.uint8(player_mask == 1) * 255
    
    # Count the number of players detected
    num_players = np.unique(kmeans.labels_).shape[0]
    
    return player_mask, num_players

# Replace 'volleyball_match.mp4' with the path to your video file
count_players("/home/gllaizz/Downloads/volleyball_match.mp4", frame_skip=500)  # Adjust frame_skip as needed


Total number of players detected: 12


In [15]:
import cv2
import numpy as np
from sklearn.cluster import KMeans

def count_players(video_path, frame_skip=1):
    cap = cv2.VideoCapture(video_path)
    
    # Initialize background subtractor
    backSub = cv2.createBackgroundSubtractorMOG2()
    
    yellow_team_count = []
    red_team_count = []
    frame_count = 0
    
    while(cap.isOpened()):
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_count += 1
        if frame_count % frame_skip != 0:
            continue  # Skip frames
        
        # Apply background subtraction
        fgMask = backSub.apply(frame)
        
        # Apply clustering to separate players based on jersey color
        yellow_mask, red_mask = cluster_players(frame)
        
        # Count the number of players detected in the yellow team
        num_yellow_players = count_players_mask(yellow_mask)
        yellow_team_count.append(num_yellow_players)
        
        # Count the number of players detected in the red team
        num_red_players = count_players_mask(red_mask)
        red_team_count.append(num_red_players)
        
        # Display the frame
        cv2.imshow('Player Count', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

    # Print the total number of players detected for each team throughout the video
    print("Total number of players in the yellow team detected:", sum(yellow_team_count))
    print("Total number of players in the red team detected:", sum(red_team_count))

def cluster_players(frame, num_clusters=2):
    # Convert frame to HSV color space
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # Define HSV ranges for yellow and red jerseys (adjust as needed)
    yellow_lower = np.array([20, 100, 100])
    yellow_upper = np.array([30, 255, 255])
    
    red_lower = np.array([0, 100, 100])
    red_upper = np.array([10, 255, 255])
    
    # Create masks for yellow and red jerseys
    yellow_mask = cv2.inRange(hsv_frame, yellow_lower, yellow_upper)
    red_mask = cv2.inRange(hsv_frame, red_lower, red_upper)
    
    return yellow_mask, red_mask

def count_players_mask(mask):
    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Count the number of players detected
    num_players = len(contours)
    
    return num_players

# Replace 'volleyball_match.mp4' with the path to your video file
count_players("/home/gllaizz/Downloads/volleyball_match.mp4", frame_skip=50)  # Adjust frame_skip as needed


Total number of players in the yellow team detected: 1640
Total number of players in the red team detected: 12806


In [29]:
import cv2
import numpy as np
from sklearn.cluster import KMeans

def count_players(video_path, frame_skip=1):
    cap = cv2.VideoCapture(video_path)
    
    # Initialize background subtractor
    backSub = cv2.createBackgroundSubtractorMOG2()
    
    yellow_team_count = []
    red_team_count = []
    frame_count = 0
    
    while(cap.isOpened()):
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_count += 1
        if frame_count % frame_skip != 0:
            continue  # Skip frames
        
        # Apply background subtraction
        fgMask = backSub.apply(frame)
        
        # Apply clustering to separate players based on jersey color
        yellow_mask, red_mask = cluster_players(frame)
        
        # Count the number of players detected in the yellow team
        num_yellow_players = count_players_mask(yellow_mask)
        yellow_team_count.append(num_yellow_players)
        
        # Count the number of players detected in the red team
        num_red_players = count_players_mask(red_mask)
        red_team_count.append(num_red_players)
        
        # Display the frame
        cv2.imshow('Player Count', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

    # Print the total number of players detected for each team throughout the video
    print("Total number of players in the yellow team detected:", sum(yellow_team_count))
    print("Total number of players in the red team detected:", sum(red_team_count))

def cluster_players(frame, num_clusters=2):
    # Convert frame to HSV color space
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # Define HSV ranges for yellow and red jerseys
    yellow_lower = np.array([20, 100, 100])
    yellow_upper = np.array([35, 255, 255])
    
    red_lower = np.array([0, 100, 100])
    red_upper = np.array([10, 255, 255])
    
    # Create masks for yellow and red jerseys
    yellow_mask = cv2.inRange(hsv_frame, yellow_lower, yellow_upper)
    red_mask = cv2.inRange(hsv_frame, red_lower, red_upper)
    
    # Apply morphological opening to reduce noise
    kernel = np.ones((5, 5), np.uint8)
    yellow_mask = cv2.morphologyEx(yellow_mask, cv2.MORPH_OPEN, kernel)
    red_mask = cv2.morphologyEx(red_mask, cv2.MORPH_OPEN, kernel)
    
    return yellow_mask, red_mask

def count_players_mask(mask):
    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Count the number of players detected
    num_players = len(contours)
    
    return num_players

# Replace 'volleyball_match.mp4' with the path to your video file
count_players("/home/gllaizz/Downloads/volleyball_match.mp4", frame_skip=)  # Adjust frame_skip as needed


Total number of players in the yellow team detected: 7
Total number of players in the red team detected: 43


In [30]:
import cv2
import numpy as np

class PlayerTracker:
    def __init__(self):
        self.tracker = cv2.TrackerCSRT_create()

    def start_tracking(self, frame, bbox):
        self.tracker.init(frame, bbox)

    def update(self, frame):
        success, bbox = self.tracker.update(frame)
        return success, bbox

class PlayerDetector:
    def __init__(self):
        self.backSub = cv2.createBackgroundSubtractorMOG2()
        self.kernel = np.ones((5, 5), np.uint8)

    def segment_color(self, frame, lower, upper):
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(hsv, lower, upper)
        return mask

    def detect_players(self, frame, yellow_lower, yellow_upper, red_lower, red_upper):
        fgMask = self.backSub.apply(frame)
        yellow_mask = self.segment_color(frame, yellow_lower, yellow_upper)
        red_mask = self.segment_color(frame, red_lower, red_upper)
        
        yellow_mask = cv2.morphologyEx(yellow_mask, cv2.MORPH_OPEN, self.kernel)
        red_mask = cv2.morphologyEx(red_mask, cv2.MORPH_OPEN, self.kernel)

        return fgMask, yellow_mask, red_mask

    def count_players(self, mask):
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        return len(contours)

def main(video_path):
    cap = cv2.VideoCapture(video_path)

    yellow_lower = np.array([20, 100, 100])
    yellow_upper = np.array([35, 255, 255])
    red_lower = np.array([0, 100, 100])
    red_upper = np.array([10, 255, 255])

    player_detector = PlayerDetector()
    player_tracker = PlayerTracker()

    yellow_count = 0
    red_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        fgMask, yellow_mask, red_mask = player_detector.detect_players(frame, yellow_lower, yellow_upper, red_lower, red_upper)

        yellow_count += player_detector.count_players(yellow_mask)
        red_count += player_detector.count_players(red_mask)

        cv2.imshow('Frame', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    print("Total number of players in the yellow team detected:", yellow_count)
    print("Total number of players in the red team detected:", red_count)

if __name__ == "__main__":
    video_path = "/home/gllaizz/Downloads/volleyball_match.mp4"
    main(video_path)

AttributeError: module 'cv2' has no attribute 'TrackerCSRT_create'

In [ ]:
import cv2
import numpy as np

class PlayerTracker:
    def __init__(self):
        self.tracker = cv2.TrackerMIL_create()

    def start_tracking(self, frame, bbox):
        self.tracker.init(frame, bbox)

    def update(self, frame):
        success, bbox = self.tracker.update(frame)
        return success, bbox

class PlayerDetector:
    def __init__(self):
        self.backSub = cv2.createBackgroundSubtractorMOG2()
        self.kernel = np.ones((5, 5), np.uint8)

    def segment_color(self, frame, lower, upper):
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(hsv, lower, upper)
        return mask

    def detect_players(self, frame, yellow_lower, yellow_upper, red_lower, red_upper):
        fgMask = self.backSub.apply(frame)
        yellow_mask = self.segment_color(frame, yellow_lower, yellow_upper)
        red_mask = self.segment_color(frame, red_lower, red_upper)
        
        yellow_mask = cv2.morphologyEx(yellow_mask, cv2.MORPH_OPEN, self.kernel)
        red_mask = cv2.morphologyEx(red_mask, cv2.MORPH_OPEN, self.kernel)

        return fgMask, yellow_mask, red_mask

    def count_players(self, mask):
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        return len(contours)

def main(video_path):
    cap = cv2.VideoCapture(video_path)

    yellow_lower = np.array([20, 100, 100])
    yellow_upper = np.array([35, 255, 255])
    red_lower = np.array([0, 100, 100])
    red_upper = np.array([10, 255, 255])

    player_detector = PlayerDetector()
    player_tracker = PlayerTracker()

    yellow_count = 0
    red_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        fgMask, yellow_mask, red_mask = player_detector.detect_players(frame, yellow_lower, yellow_upper, red_lower, red_upper)

        yellow_count += player_detector.count_players(yellow_mask)
        red_count += player_detector.count_players(red_mask)

        cv2.imshow('Frame', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    print("Total number of players in the yellow team detected:", yellow_count)
    print("Total number of players in the red team detected:", red_count)

if __name__ == "__main__":
    video_path = "/home/gllaizz/Downloads/volleyball_match.mp4"
    main(video_path)
